In [6]:
using LinearAlgebra

function read_matrix(path::AbstractString)
    lines = readlines(path)
    rows = [parse.(Float64, split(line)) for line in lines if !isempty(strip(line))]
    n = length(rows)
    m = length(rows[1])
    @assert n == m "Матрица должна быть квадратной"
    A = Matrix{Float64}(undef, n, n)
    for i in 1:n, j in 1:n
        A[i, j] = rows[i][j]
    end
    return A
end

function danilevsky_algorithm(A::Matrix{Float64})
    n = size(A, 1)
    B = Matrix{Float64}(I, n, n)
    P = copy(A)

    for k in n:-1:2
        B_k = Matrix{Float64}(I, n, n)
        for j in 1:n
            if j != k - 1
                B_k[k - 1, j] = -P[k, j] / P[k, k - 1]
            else
                B_k[k - 1, j] = 1.0 / P[k, k - 1]
            end
        end

        B_k_inverted = Matrix{Float64}(I, n, n)
        for j in 1:n
            B_k_inverted[k - 1, j] = P[k, j]
        end

        P = B_k_inverted * P * B_k
        B = B * B_k
    end

    return B, P
end

function union_intervals(intervals::Vector{Vector{Float64}})
    isempty(intervals) && return Vector{Vector{Float64}}()
    s = sort(intervals, by = x -> x[1])
    merged = [copy(s[1])]
    for v in s[2:end]
        last = merged[end]
        if last[2] >= v[1]
            last[2] = max(last[2], v[2])
        else
            push!(merged, copy(v))
        end
    end
    merged
end

function gershgorin_intervals_raw(A::Matrix{Float64})
    n = size(A, 1)
    ivs = Vector{Vector{Float64}}(undef, n)
    for i in 1:n
        r = sum(abs.(A[i, :])) - abs(A[i, i])
        c = A[i, i]
        ivs[i] = [c - r, c + r]
    end
    ivs
end

gershgorin_intervals(A::Matrix{Float64}) = union_intervals(gershgorin_intervals_raw(A))

find_equation_coeffs(D::Matrix{Float64}) = vcat(1.0, -D[1, :])

function poly_eval(coeffs::AbstractVector{<:Real}, x::Real)
    acc = 0.0
    for c in coeffs
        acc = acc * x + c
    end
    acc
end

function find_real_roots_on_intervals(coeffs::Vector{Float64},
                                      intervals::Vector{Vector{Float64}};
                                      step::Real = 1e-2, tol::Real = 1e-8)
    roots = Float64[]
    for I in intervals
        a, b = I
        x = a
        prev_y = poly_eval(coeffs, x)
        while x < b - step / 2
            x_next = min(x + step, b)
            y_next = poly_eval(coeffs, x_next)
            if prev_y == 0.0
                push!(roots, x)
            elseif y_next == 0.0
                push!(roots, x_next)
            elseif prev_y * y_next < 0
                left, right = x, x_next
                yl = prev_y
                while right - left > tol
                    mid = (left + right) / 2
                    ym = poly_eval(coeffs, mid)
                    if yl * ym <= 0
                        right = mid
                    else
                        left = mid
                        yl = ym
                    end
                end
                push!(roots, (left + right) / 2)
            end
            x = x_next
            prev_y = y_next
        end
    end
    sort!(unique!(round.(roots; digits = 10)))
    roots
end

function danilevsky_eigenvalues(A::Matrix{Float64};
                                step::Real = 1e-3,
                                tol::Real = 1e-10)
    _, Df = danilevsky_algorithm(A)
    coeffs = find_equation_coeffs(Df)
    intervals = gershgorin_intervals(A)
    eig_vals = find_real_roots_on_intervals(coeffs, intervals; step = step, tol = tol)
    return eig_vals
end

function main()
    filename = "matrix.txt"
    A = read_matrix(filename)
    println("Матрица A из файла:")
    show(stdout, "text/plain", A)
    println("\n")
    eig_vals = danilevsky_eigenvalues(A)
    println(eig_vals)
end

main()

Матрица A из файла:
4×4 Matrix{Float64}:
 260.39  167.32  230.97  134.64
 167.32  125.43  146.51   66.11
 230.97  146.51  211.38  123.79
 134.64   66.11  123.79  100.94

[1.8405567906769327, 3.9113661266833457, 46.23723138184363, 646.1508457008049]
